### Load necessary packages

In [ ]:
import torch
import torch.optim as optim

from dataset import FontsLoader
from models import VAE

import matplotlib.pyplot as plt
%matplotlib inline

### Setup the device on which to train

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

### Load the data

In [ ]:
set_loader = FontsLoader.get_set_loader()

### Build the Net

In [ ]:
net = VAE(
    latent_dim=64,
    beta=5,
    in_channels=1,
    num_hiddens=128,
    num_res_hiddens=32,
    num_res_layers=2,
    out_channels=1,
).to(device)

## Train loop

In [ ]:
def train(epochs=10):
    print('='*10, end='')
    print(' TRAIN', end=' ') 
    print('='*10, end='\n\n')
    net.train()

    for epoch in range(1, epochs+1):
        running_loss = 0

        for i, batch in enumerate(set_loader, 1):
            images = batch['image']
            images = images.to(device)
            
            # Zero grad
            optimizer.zero_grad()
            
            # Forward
            mu, logvar, z, recon = net(images)
            # Compute Loss
            loss_value = net.loss_function(images, recon, mu, logvar)
            running_loss += loss_value.item()
            # Backward
            loss_value.backward()
            # Update
            optimizer.step()

            if i % 5 == 0:
                print(f'==> EPOCH[{epoch}]({i}/{len(set_loader)}): LOSS: {loss_value.item()}')
            
        print(f'=====> EPOCH[{epoch}] Completed: Avg. LOSS: {running_loss/len(set_loader)}')
        print()
        
    net.eval()

### Init optimizer and train

In [ ]:
optimizer = optim.Adam(net.parameters(), lr=1e-6)

In [ ]:
train()

In [ ]:
torch.save(net.state_dict(), open('vae_30_epochs_loss_14.pth', 'wb'))